In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import datetime

# import finlib

The below code snippets no longer works because yahoo finance has changed their API to not allow calls to it from anything other that their own website. I am leaving the code here for reference.

In [ ]:
base_url = 'https://www.nseindia.com'
response = requests.get(base_url, headers={'User-Agent': 'Mozilla/5.0'})

# THe NSE site requires a cookie to be passed to query the API along with a user agent.
cookies = response.cookies
request_headers = {'User-Agent': 'Mozilla/5.0'}

yahoo_response = requests.get(base_url + '/api/equity-stockIndices?index=NIFTY%2050', headers=request_headers, cookies=cookies).json()

yahoo_response

data = yahoo_response['data']

symbols = [(i['symbol'], i['symbol'] + '.NS') for i in data if i['symbol'] != 'NIFTY 50']

symbol_dfs = []
start_time = int(datetime.datetime(2024,1,1,0,0,0).timestamp())
end_time = int(datetime.datetime.now().timestamp())

for symbol, name in symbols:
    url = f'https://query1.finance.yahoo.com/v7/finance/download/{name}?period1={start_time}&period2={end_time}&interval=1d&events=history&includeAdjustedClose=true'
    d = pd.read_csv(url, usecols=['Date', 'Adj Close'], parse_dates=['Date'], index_col='Date')
    d.index = d.index.date
    symbol_dfs.append((symbol, d))

nifty50_df = pd.DataFrame()

for df in symbol_dfs:
    nifty50_df[df[0]] = df[1]['Adj Close']

nifty50_df.index = pd.to_datetime(nifty50_df.index)

In the next step we are going to get the sector and the industry of the companies. This will get used at a later stage to filter the master dataset.

In [156]:
import yfinance as yf

company_sector_industry = pd.DataFrame(columns=['Symbol', 'Sector', 'Industry'])

for symbol, yfin_name in symbols:
    stock_info = yf.Ticker(yfin_name).info

    temp = pd.DataFrame({
        'Symbol': symbol,
        'yfin_name': yfin_name,
        'Sector': stock_info.get('sector', 'N/A'),
        'Industry': stock_info.get('industry', 'N/A')
    }, index=[0])

    company_sector_industry = pd.concat([company_sector_industry, temp], ignore_index=True)

company_sector_industry.set_index('Symbol', inplace=True)
    

In [ ]:
company_sector_industry

In [158]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_nifty50_data = scaler.fit_transform(nifty50_df)
scaled_df = pd.DataFrame(scaled_nifty50_data, index=nifty50_df.index, columns=nifty50_df.columns)

In [ ]:
scaled_df

In [160]:
cols = [ column for column in nifty50_df.columns if company_sector_industry.loc[column].Sector == 'Consumer Defensive' ]